# Evaluación 4 – Machine Learning (MLOps)
## Aplicación de MLOps al modelo de la Evaluación 3 (MNIST)

**Estudiante:** Jorge Barrios
**Fecha:** 05-12-2025  

En esta evaluación, aplico conceptos de **MLOps** al modelo de clasificación de dígitos MNIST utilizado en la Evaluación 3.

El objetivo es:

- Determinar qué elementos del proceso de ML se deben **monitorear** y **versionar**.
- Aplicar **versionamiento** y **trazabilidad** sobre datos, código, modelos y métricas.
- Identificar qué tareas del flujo original pueden ser **automatizadas**.
- Implementar una **automatización básica** mediante funciones y scripts que permitan repetir el entrenamiento de forma reproducible.

El dataset utilizado es **MNIST**, cargado desde `keras.datasets.mnist`, tal como en la Evaluación 3:
- Imágenes de dígitos escritos a mano (28x28 píxeles, en escala de grises).
- 10 clases (dígitos de 0 a 9).


In [1]:
!pip install mlflow scikit-learn tensorflow pandas numpy matplotlib -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 753.9/753.9 kB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 15.3 MB/s eta 0:00:00


In [2]:
import hashlib
from tensorflow import keras
import numpy as np
import pandas as pd

def cargar_dataset():
    """
    Carga el dataset MNIST (el mismo que en la Evaluación 3) y lo transforma
    en un DataFrame:
      - X: imágenes 28x28 aplanadas a 784 columnas.
      - y: dígitos (0-9) en la columna 'target'.
      - Normalización de pixeles a [0, 1].
    """
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

    # Unir train + test para tener un único dataset
    X = np.concatenate([x_train, x_test], axis=0)
    y = np.concatenate([y_train, y_test], axis=0)

    # Aplanar imágenes 28x28 -> 784 y normalizar
    X = X.reshape((X.shape[0], -1)).astype("float32") / 255.0

    # DataFrame: 784 columnas de features + 1 columna 'target'
    df = pd.DataFrame(X)
    df["target"] = y

    return df

def resumir_dataset(df: pd.DataFrame):
    print("Shape:", df.shape)
    print("\nTipos de datos (primeras columnas):")
    print(df.dtypes.head())
    print("\nValores nulos por columna (primeras columnas):")
    print(df.isna().sum().head())

def hash_dataframe(df: pd.DataFrame) -> str:
    """
    Calcula un hash MD5 del contenido del DataFrame para simular
    versionamiento de datos (detecta cambios en el dataset).
    """
    df_bytes = pd.util.hash_pandas_object(df, index=True).values
    return hashlib.md5(df_bytes).hexdigest()

def validar_dataset(df: pd.DataFrame):
    """
    Validación automática básica del dataset.
    - Verifica que no esté vacío.
    - Verifica que no haya valores nulos.
    - Verifica que las features estén en el rango [0, 1].
    """
    if df.empty:
        raise ValueError("Dataset vacío")

    if df.isna().sum().sum() > 0:
        print("Advertencia: hay valores nulos, se requiere imputación.")
    else:
        print("OK: no hay valores nulos.")

    # Chequeo simple de rangos (features deben estar entre 0 y 1)
    num_cols = df.drop(columns=["target"])
    min_val = num_cols.min().min()
    max_val = num_cols.max().max()

    if min_val < 0 or max_val > 1:
        print(f"Advertencia: hay valores fuera de [0, 1]. min={min_val}, max={max_val}")
    else:
        print("OK: todas las features están en [0, 1].")

    print("Validación básica completada.")


In [3]:
df = cargar_dataset()
resumir_dataset(df)
dataset_hash = hash_dataframe(df)
print("\nHash del dataset:", dataset_hash)
validar_dataset(df)


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Shape: (70000, 785)

Tipos de datos (primeras columnas):
0    float32
1    float32
2    float32
3    float32
4    float32
dtype: object

Valores nulos por columna (primeras columnas):
0    0
1    0
2    0
3    0
4    0
dtype: int64

Hash del dataset: 7cd394616d2ece8836deb9648901b599
OK: no hay valores nulos.
OK: todas las features están en [0, 1].
Validación básica completada.


## 1. Elementos a monitorear y versionar

En el contexto del modelo MNIST de la Evaluación 3, decido monitorear y versionar los siguientes elementos:

### 1.1 Datos (Dataset MNIST)

- **Origen de datos**: `keras.datasets.mnist`.
- **Tamaño**: número de filas y columnas (`df.shape`).
- **Esquema**: nombres y tipos de las columnas.
- **Transformaciones**: aplanamiento de las imágenes (28x28 → 784) y normalización a [0, 1].
- **Hash del dataset**: se calcula un `hash` (MD5) del DataFrame para detectar cambios en el contenido.

### 1.2 Modelo

- Tipo de modelo: en este caso uso un **RandomForestClassifier** sobre las features aplanadas, como ejemplo de modelo clásico aplicado a MNIST.
- **Hiperparámetros** principales:
  - `n_estimators`
  - `max_depth`
  - `test_size` usado en el split
- Versión lógica del modelo: a través de **runs** en MLflow (por ejemplo `run_id`, nombre del experimento).

### 1.3 Métricas

- Métricas principales del modelo:
  - `accuracy`
  - `f1_score` (macro o weighted, según configuración)
- Otros outputs:
  - Reporte de clasificación (precision, recall) que se puede guardar como artefacto textual.

### 1.4 Ambiente y dependencias

- Versión de Python.
- Versiones de librerías: `scikit-learn`, `pandas`, `numpy`, `tensorflow`, `mlflow`, etc.
- Archivo `requirements.txt` con las dependencias mínimas.

### 1.5 Preprocesamiento

- Pasos de preparación de los datos:
  - Reescalado de features (si corresponde).
  - División train/test.
- En este ejemplo, el RandomForest no requiere escalado explícito, pero dejo preparado el pipeline con posibilidad de incluir `StandardScaler`.

### 1.6 Logs y ejecución

- Fecha y hora de entrenamiento.
- Identificador del experimento (run) en MLflow.
- Mensajes de logging impresos durante el entrenamiento (accuracy, F1, etc.).

Para la **trazabilidad completa**:
- Uso **Git** para versionar el código del notebook y los scripts (`main_pipeline.py`, `run_pipeline.py`).
- Uso **MLflow** para registrar parámetros, métricas, hash del dataset y artefactos del modelo.


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

def dividir_dataset(df: pd.DataFrame, test_size=0.2):
    """
    Separa el dataset en train y test.
    """
    X = df.drop(columns=["target"])
    y = df["target"]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        test_size=test_size,
        random_state=42,
        stratify=y
    )

    return X_train, X_test, y_train, y_test

def construir_pipeline(n_estimators=100, max_depth=None):
    """
    Construye un pipeline simple:
      - (Opcional) escalado.
      - RandomForestClassifier.
    """
    pipeline = Pipeline(steps=[
        ("scaler", StandardScaler(with_mean=False)),  # with_mean=False por sparse posible
        ("clf", RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            random_state=42,
            n_jobs=-1
        ))
    ])
    return pipeline


In [5]:
import mlflow
import mlflow.sklearn
import os
from datetime import datetime

# Configuración de MLflow (tracking local en la carpeta mlruns)
tracking_uri = os.path.join(os.getcwd(), "mlruns")
mlflow.set_tracking_uri(f"file:{tracking_uri}")
mlflow.set_experiment("eva4_mnist_mlops")

def entrenar_y_registrar_modelo(
    n_estimators=100,
    max_depth=None,
    test_size=0.2,
):
    """
    Entrena el modelo sobre MNIST, valida el dataset,
    registra todo en MLflow y devuelve el modelo + métricas.
    """
    # Cargar y validar datos
    df = cargar_dataset()
    validar_dataset(df)

    # División train/test
    X_train, X_test, y_train, y_test = dividir_dataset(df, test_size=test_size)

    # Comenzar un run de MLflow
    with mlflow.start_run(run_name=f"mnist_run_{datetime.now().strftime('%Y%m%d_%H%M%S')}") as run:
        model = construir_pipeline(n_estimators=n_estimators, max_depth=max_depth)

        # Entrenamiento
        model.fit(X_train, y_train)

        # Predicciones
        y_pred = model.predict(X_test)

        # Métricas
        acc = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average="macro")

        # ---- Log de parámetros ----
        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)
        mlflow.log_param("test_size", test_size)
        mlflow.log_param("dataset_hash", hash_dataframe(df))

        # ---- Log de métricas ----
        mlflow.log_metric("accuracy", acc)
        mlflow.log_metric("f1_macro", f1)

        # ---- Log de artefactos ----
        # Guardar el modelo
        mlflow.sklearn.log_model(model, artifact_path="modelo")

        # Guardar un reporte de clasificación
        report = classification_report(y_test, y_pred)
        report_path = "reporte_clasificacion.txt"
        with open(report_path, "w") as f:
            f.write("Reporte de clasificación (MNIST):\n")
            f.write(report)
            f.write("\n")
            f.write(f"Accuracy: {acc:.4f}\n")
            f.write(f"F1 macro: {f1:.4f}\n")
        mlflow.log_artifact(report_path)

        print("Run ID:", run.info.run_id)
        print(f"Accuracy: {acc:.4f}")
        print(f"F1-score (macro): {f1:.4f}")

    return model, acc, f1


/usr/local/lib/python3.12/dist-packages/mlflow/tracking/_tracking_service/utils.py:177: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance.
  return FileStore(store_uri, store_uri)
2025/12/05 18:32:00 INFO mlflow.tracking.fluent: Experiment with name 'eva4_mnist_mlops' does not exist. Creating a new experiment.


In [6]:
modelo, acc, f1 = entrenar_y_registrar_modelo(
    n_estimators=120,
    max_depth=20,
    test_size=0.2
)

print("\nEntrenamiento completado.")
print(f"Accuracy final: {acc:.4f}")
print(f"F1-score macro final: {f1:.4f}")


OK: no hay valores nulos.
OK: todas las features están en [0, 1].
Validación básica completada.


2025/12/05 18:33:09 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Run ID: 434f4667783141f89c95d977365a0516
Accuracy: 0.9669
F1-score (macro): 0.9666

Entrenamiento completado.
Accuracy final: 0.9669
F1-score macro final: 0.9666


## 2. Tareas a automatizar (poda de actividades manuales)

Antes de aplicar MLOps, el flujo típico del modelo de MNIST incluía varias tareas manuales:

1. **Carga del dataset**:
   - Ejecutar manualmente el código de `keras.datasets.mnist.load_data()`.
   - Aplicar manualmente la transformación de las imágenes.

2. **Preprocesamiento y validación**:
   - Revisar manualmente si el dataset tiene nulos o valores fuera de rango.
   - Dividir manualmente en train/test.

3. **Entrenamiento del modelo**:
   - Escribir el código del modelo y ejecutar las celdas a mano.
   - Ajustar hiperparámetros probando distintas celdas.

4. **Evaluación y métricas**:
   - Calcular accuracy/F1 manualmente.
   - Mirar los resultados sin registrarlos de forma estructurada.

5. **Guardado del modelo y resultados**:
   - Guardar el modelo con `joblib`/`pickle` a mano.
   - Anotar resultados en otro lado (documento, Excel, etc.).

### 2.1. Decisiones de automatización

Para reducir errores y hacer el proceso reproducible, automatizo:

- **Validación del dataset**:
  - La función `validar_dataset(df)` se ejecuta siempre antes de entrenar.

- **Entrenamiento completo en una sola función**:
  - `entrenar_y_registrar_modelo(...)` encapsula:
    - Carga de datos.
    - Validación.
    - División train/test.
    - Entrenamiento.
    - Cálculo de métricas.
    - Registro en MLflow.

- **Versionamiento y trazabilidad**:
  - Cada corrida queda registrada en **MLflow** con parámetros, métricas y `dataset_hash`.
  - El modelo entrenado se guarda como **artefacto**.

- **Script ejecutable único**:
  - Defino un script `run_pipeline.py` que permite entrenar el modelo ejecutando un solo comando:
    - `python run_pipeline.py`

Con estas automatizaciones, el entrenamiento del modelo MNIST deja de depender de pasos manuales dispersos y pasa a ser un pipeline reproducible y auditable.


In [7]:
%%writefile main_pipeline.py
import os
import hashlib
from datetime import datetime

import numpy as np
import pandas as pd
from tensorflow import keras

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

import mlflow
import mlflow.sklearn


def cargar_dataset():
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
    X = np.concatenate([x_train, x_test], axis=0)
    y = np.concatenate([y_train, y_test], axis=0)

    X = X.reshape((X.shape[0], -1)).astype("float32") / 255.0

    df = pd.DataFrame(X)
    df["target"] = y
    return df


def hash_dataframe(df: pd.DataFrame) -> str:
    df_bytes = pd.util.hash_pandas_object(df, index=True).values
    return hashlib.md5(df_bytes).hexdigest()


def validar_dataset(df: pd.DataFrame):
    if df.empty:
        raise ValueError("Dataset vacío")

    if df.isna().sum().sum() > 0:
        print("Advertencia: hay valores nulos, se requiere imputación.")
    else:
        print("OK: no hay valores nulos.")

    num_cols = df.drop(columns=["target"])
    min_val = num_cols.min().min()
    max_val = num_cols.max().max()

    if min_val < 0 or max_val > 1:
        print(f"Advertencia: hay valores fuera de [0, 1]. min={min_val}, max={max_val}")
    else:
        print("OK: todas las features están en [0, 1].")

    print("Validación básica completada.")


def dividir_dataset(df: pd.DataFrame, test_size=0.2):
    X = df.drop(columns=["target"])
    y = df["target"]
    X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        test_size=test_size,
        random_state=42,
        stratify=y
    )
    return X_train, X_test, y_train, y_test


def construir_pipeline(n_estimators=100, max_depth=None):
    pipeline = Pipeline(steps=[
        ("scaler", StandardScaler(with_mean=False)),
        ("clf", RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            random_state=42,
            n_jobs=-1
        ))
    ])
    return pipeline


# Configuración de MLflow
tracking_uri = os.path.join(os.getcwd(), "mlruns")
mlflow.set_tracking_uri(f"file:{tracking_uri}")
mlflow.set_experiment("eva4_mnist_mlops_script")


def entrenar_y_registrar_modelo(
    n_estimators=100,
    max_depth=None,
    test_size=0.2,
):
    df = cargar_dataset()
    validar_dataset(df)

    X_train, X_test, y_train, y_test = dividir_dataset(df, test_size=test_size)

    with mlflow.start_run(run_name=f"mnist_run_{datetime.now().strftime('%Y%m%d_%H%M%S')}") as run:
        model = construir_pipeline(n_estimators=n_estimators, max_depth=max_depth)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        acc = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average="macro")

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)
        mlflow.log_param("test_size", test_size)
        mlflow.log_param("dataset_hash", hash_dataframe(df))

        mlflow.log_metric("accuracy", acc)
        mlflow.log_metric("f1_macro", f1)

        report = classification_report(y_test, y_pred)
        report_path = "reporte_clasificacion.txt"
        with open(report_path, "w") as f:
            f.write("Reporte de clasificación (MNIST):\n")
            f.write(report)
            f.write("\n")
            f.write(f"Accuracy: {acc:.4f}\n")
            f.write(f"F1 macro: {f1:.4f}\n")
        mlflow.log_artifact(report_path)

        mlflow.sklearn.log_model(model, artifact_path="modelo")

        print("Run ID:", run.info.run_id)
        print(f"Accuracy: {acc:.4f}")
        print(f"F1-score (macro): {f1:.4f}")

    return model, acc, f1


Writing main_pipeline.py


In [8]:
%%writefile run_pipeline.py
from main_pipeline import entrenar_y_registrar_modelo

if __name__ == "__main__":
    # Hiperparámetros por defecto (podrían venir de variables de entorno)
    entrenar_y_registrar_modelo(
        n_estimators=150,
        max_depth=25,
        test_size=0.2,
    )


Writing run_pipeline.py


In [9]:
!python run_pipeline.py


2025-12-05 18:39:50.373693: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764959990.451266    3301 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764959990.475299    3301 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764959990.538108    3301 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764959990.538212    3301 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764959990.538227    3301 computation_placer.cc:177] computation placer alr

In [10]:
%%writefile requirements.txt
mlflow
scikit-learn
tensorflow
pandas
numpy
matplotlib


Writing requirements.txt


In [11]:
import os
os.makedirs(".github/workflows", exist_ok=True)


In [12]:
%%writefile .github/workflows/train.yml
name: train-mnist-model

on:
  push:
    branches: [ main ]
  workflow_dispatch:

jobs:
  train:
    runs-on: ubuntu-latest

    steps:
      - name: Checkout repository
        uses: actions/checkout@v4

      - name: Set up Python
        uses: actions/setup-python@v5
        with:
          python-version: '3.10'

      - name: Install dependencies
        run: |
          pip install -r requirements.txt

      - name: Run training pipeline
        run: |
          python run_pipeline.py


Writing .github/workflows/train.yml


In [13]:
!git init
!git status
!git add main_pipeline.py run_pipeline.py requirements.txt .github/train.yml 2>/dev/null || echo "Archivos listos para versionar."


hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/
On branch master

No commits yet

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.config/
	.github/
	__pycache__/
	main_pipeline.py
	mlruns/
	reporte_clasificacion.txt
	requirements.txt
	run_pipeline.py
	sample_data/

nothing added to commit but untracked files present (use "git add" to track)
Archivos listos para versionar.


In [14]:
# Commit simple (puede fallar si no configuras user.name / user.email, es normal en Colab)
!git commit -m "EVA4 - Pipeline MNIST con MLOps" || echo("Commit no ejecutado (falta configurar git), pero el flujo de versionamiento está descrito.")


/bin/bash: -c: line 1: syntax error near unexpected token `"Commit no ejecutado (falta configurar git), pero el flujo de versionamiento está descrito."'
/bin/bash: -c: line 1: `git commit -m "EVA4 - Pipeline MNIST con MLOps" || echo("Commit no ejecutado (falta configurar git), pero el flujo de versionamiento está descrito.")'


## 3. Conclusiones

En esta Evaluación 4 apliqué prácticas de **MLOps** al modelo de clasificación de dígitos MNIST utilizado en la Evaluación 3:

- **Elementos monitoreados y versionados**:
  - Datos (MNIST) con validación básica y cálculo de `hash` para detectar cambios.
  - Modelo (RandomForest sobre imágenes aplanadas) con hiperparámetros registrados.
  - Métricas (accuracy y F1-score macro) para comparar experimentos.
  - Ambiente y dependencias (requirements.txt).

- **Versionamiento y trazabilidad**:
  - Uso de **MLflow** como sistema de tracking local (experimentos, parámetros, métricas y artefactos).
  - Uso de **Git** para versionar código y archivos auxiliares (`main_pipeline.py`, `run_pipeline.py`, `requirements.txt`, workflow de GitHub Actions).

- **Poda y automatización de tareas**:
  - Encapsulé el proceso completo de entrenamiento en `entrenar_y_registrar_modelo`, reduciendo pasos manuales dispersos.
  - Definí un script ejecutable `run_pipeline.py` para poder lanzar el entrenamiento con un único comando.
  - Preparé un workflow de **CI/CD (GitHub Actions)** que instala dependencias y ejecuta el pipeline automáticamente ante un `push` o disparo manual.

Con esto, el proceso de entrenamiento y evaluación del modelo MNIST deja de ser un conjunto de celdas aisladas y pasa a ser un **pipeline reproducible, trazable y automatizable**, alineado con los objetivos de MLOps propuestos en la evaluación.
